## Youtube Data API

Add Youtube Functionality to your site

Search for content

Get the Youtube Channel ID: https://www.streamweasels.com/tools/youtube-channel-id-and-user-id-convertor/

## Features

- Control the video length

In [ ]:
!pip install --upgrade google-api-python-client
!pip install --upgrade google-auth-oauthlib google-auth-httplib2
!pip install schedule
!pip install ffmpeg-python
!pip install yt-dlp
!pip install unidecode

In [ ]:
import pandas as pd
import requests
import json

In [8]:
import os
import googleapiclient.discovery
from pprint import pprint 
import schedule
from datetime import datetime, timedelta
import time 
from Youtube import download_audio, download_video
from dotenv import load_dotenv

API_KEY = os.getenv("API_KEY")
API_SERVICE_NAME = "youtube"
API_VERSION = "v3"

CHANNEL_ID = "UCabsTV34JwALXKGMqHpvUiA" # vtv24h
MAX_RESULT = 50

def get_channel_info():
    youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)
    request = youtube.channels().list(part="snippet", id=CHANNEL_ID)
    response = request.execute()

    pprint(response)

def get_channel_statistics():
    youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)
    request = youtube.channels().list(part="statistics", id=CHANNEL_ID)
    response = request.execute()

    pprint(response)

def create_video_url(video_id):
    return f"https://www.youtube.com/watch?v={video_id}"


def get_latest_video():
    print("Fetching latest video ")
    youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)

    yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d') + "T00:00:00Z"

    video_links = []

    try:
        # Fetch the latest video published after yesterday
        request = youtube.search().list(
            q="",
            channelId=CHANNEL_ID,
            maxResults=MAX_RESULT,
            order="date",
            publishedAfter=yesterday,
            type="video",
            part="snippet"
        )
        response = request.execute()
    
        print("Total number of results: ", response['pageInfo']['totalResults'])
    
        # Parse the items
        for item in response['items']:
            video_url = create_video_url(item['id']['videoId'])
            title = item['snippet']['title']
            desc = item['snippet']['description']
            publish_time = item['snippet']['publishedAt']
            print(video_url, title, desc, publish_time)
            video_links.append(video_url)
            
    except Exception as e:
        print(f"Error: {str(e)}")
    
    return response

    # print("Download videos")
    # download_video(video_links)

# Schedule the task to run every day at 7:00 AM 
# schedule.every().day.at("23:59").do(get_latest_video)
# schedule.every(60).seconds.do(get_latest_video)

# get_channel_statistics()
# response = get_latest_video()

# while True:
#     schedule.run_pending()
#     time.sleep(1)

In [9]:
response = get_latest_video()

Fetching latest video 
Total number of results:  37
https://www.youtube.com/watch?v=fmDoMNCZWRk Thời sự toàn cảnh tối 9/11: Vì sao phải tăng giá điện? | VTV24 Những nội dung sẽ có trong Thời sự toàn cảnh tối 9/11: 00:00 Tin chính 00:43 Làm khống hồ sơ nghiệm thu xe cơ giới 01:27 Bị ... 2023-11-09T14:00:11Z
https://www.youtube.com/watch?v=MT6czwLtz6A Giá bán lẻ điện bình quân tăng 4,5% | VTV24 Từ hôm nay, giá bán lẻ điện bình quân tăng 4,5%. Mức tăng giá này ảnh hưởng thế nào tới người thu nhập thấp, thu nhập trung ... 2023-11-09T12:02:49Z
https://www.youtube.com/watch?v=UB0Etn8h7Ek Nhiều cửa hàng bách hóa ở Thượng Hải giảm giá dịp lễ độc thân | VTV24 Ngày 11/11 là ngày độc thân ở Trung Quốc. Bốn số 1 được giải nghĩa giống như bốn cành cây trơ trụi, một cách diễn đạt dành ... 2023-11-09T10:00:21Z
https://www.youtube.com/watch?v=BlXKi8TokAU Câu chuyện quốc tế: Sống trong thành phố tương lai | VTV24 Từ việc tái sử dụng những công trình cũ, đến việc sử dụng những công nghệ giảm phát thải..

In [10]:
len(response['items'])

37

## Download Video/Audio

In [ ]:
from Youtube import download_audio, download_video
URLS = ["https://www.youtube.com/watch?v=mklY28SqTzc"]
download_audio(URLS)
download_video(URLS)